# Basic Spatially-Extended Stochastic Lotka-Volterra Model

In this first example, we want to focus only on a simple non-spatial system. In essence, this is equivalent to a lattice with only a single, isolated lattice site.

We need to install the python packages we need to run the simulations:

In [ ]:
%pip install git+https://github.com/ulido/pypop matplotlib numpy pandas ipywidgets

To start, we need to load the `pypop` library. We only load what we need so not to clutter our environment.

In [ ]:
from pypop import World, Species, BirthReaction, DeathReaction, PredationBirthReaction, Hop

Next, we construct the world. This involves setting up the species (the predator A and the prey B) and the reactions (B can give birth, A can die and predate upon B). The following are the parameters of the simulation:
* `size`: This is the size of the 2D lattice and is specified as a tuple of two integers (the width and the height) like so `(256, 256)`. It is a good idea to start small since larger lattices take more memory and more computational time to simulate.
* `sigma`: This is the prey (species "B") birth rate. It regulates how quickly prey will reproduce.
* `mu`: This is the predator (species "A") death rate. It regulates how quickly predators are removed from the system.
* `lam`: This is the predation rate. It regulates how efficiently predators (species "A") consume prey (species "B") and reproduce at the same time.
* `hA` and `hB`: This is the hop rate, which in typically is left at a value of `1` since this will only rescale time. It could be varied for each species separately, such that prey e.g. more static than predators or vice-versa.
* `rhoA` and `rhoB`: This is the initial density of the predators (species "A") and prey (species "B"). It specifies how many occupants of each species will be present on each lattice site on average.
* `T`: Specifies the end time of the simulation, i.e. how many Monte Carlo steps will be performed.

In [ ]:
# The simulation parameters
size = (100, 100)
sigma: float = 0.1
mu: float = 0.1
lam: float = 0.1
hA: float = 1.0
hB: float = 1.0
rhoA: float = 0.01
rhoB: float = 1.0
T: int = 1000

# Predator and prey species
A = Species("A")
B = Species("B")

# The reactions
reactions = {
    A: [
        PredationBirthReaction(A, B, lam),
        DeathReaction(A, mu),
    ],
    B: [
        BirthReaction(B, sigma),
    ]
}

world = World(
    size=size,
    initial_densities={B: rhoB, A: rhoA},
    hops={A: Hop(A, hA), B: Hop(B, hB)},
    reactions=reactions,
)

In [ ]:
from tqdm.auto import tqdm
numbers = []
for _ in tqdm(range(T), smoothing=0):
    world.step()
    numbers.append(world.asarrays())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

N = size[0] * size[1]
df = pd.DataFrame([{species: arr.sum()/N for species, arr in entry.items()} for entry in numbers])

fig, ax = plt.subplots(1, 1, figsize=(5, 4), dpi=150)
ax.plot(df.A, color="red", label="A")
ax.plot(df.B, color="blue", label="B")
ax.set_yscale("log", base=10)
ax.set_ylabel("Species density [occupants/site]")
ax.set_xlabel("Time [MC steps]")


In [ ]:
from ipywidgets import interact
import numpy as np
from IPython.display import display

fig = plt.Figure(figsize=(8, 4), dpi=150)
ax = fig.add_axes([0, 0, 0.5, 1])
plot = fig.add_axes([0.6, 0.1, 0.4, 0.8])

plot.plot(df.A, color="red", label="A")
plot.plot(df.B, color="blue", label="B")
plot.set_yscale("log", base=10)
plot.set_ylabel("Species density [occupants/site]")
plot.set_xlabel("Time [MC steps]")

mappable = None
vline = None
def show_image(MC_step=0):
    global mappable, vline
    arrays = numbers[MC_step]
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    image[:, :, 0] = 255*(arrays["A"] > 0)
    image[:, :, 2] = 255*(arrays["B"] > 0)
    if mappable is None:
        mappable = ax.imshow(image)
        vline = plot.axvline(0, color="black")
    else:
        mappable.set_data(image)
        vline.set_xdata([MC_step])
    display(fig)

ax.axis('off')
interact(show_image, MC_step=(0, len(numbers)-1))
None